In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,'n Sync,This Is Where The Party's At,Yeah \r\nThis is where's the party's at \r\n...
1,Phil Collins,Captain Planet Theme Song,Earth! \r\nFire! \r\nWind! \r\nWater! \r\n...
2,Alan Parsons Project,I Wouldn't Want To Be Like You,If I had a mind to \r\nI wouldn't wanna think...
3,Kenny Chesney,Scare Me,I'm not afraid of lonely \r\nI've gotten good...
4,Christmas Songs,"Percy, The Puny Poinsettia",Percy the puny poinsettia \r\nIs hanging his ...
5,Yo La Tengo,Last Days Of Disco,Saw you at a party \r\nYou asked me to dance ...
6,Oingo Boingo,We Close Our Eyes,We close our eyes \r\nWe close our eyes and d...
7,Metallica,It's Electric,(Originally recorded by Diamond Head) \r\n \...
8,Ozzy Osbourne,Rocky Mountain Way,Spent the last year \r\nRocky mountain way \...
9,Don Henley,You Don't Know Me At All,I woke up this morning \r\nWith an attitude ...


In [9]:
df['text'][0]

"Yeah  \r\nThis is where's the party's at  \r\n(This is where's the party's at)  \r\nCome on babe  \r\nThis is where's the party's at  \r\n(this is where's the party's at)  \r\nThis is where's the party's at  \r\n  \r\nMonday morning till Friday afternoon  \r\nStressin' me out  \r\nBut tonight the weekend comes around  \r\nAnd its time to break out  \r\n  \r\nEveryone needs some time,  \r\nTime to get away,  \r\nBaby come free your mind  \r\n(Free your mind and free your soul)  \r\nThis is your night to shine If you want to play  \r\nBaby Its okay (Its okay to loose control now)  \r\n  \r\nIf you want to grove now,  \r\nThis is where's the party's at  \r\nGotta make you move now,  \r\nThis is where's the party's at  \r\nEverybody gettin in the zone,  \r\nThis is where's the party's at  \r\nEverybody come on.  \r\nThis is where's the party's at  \r\n  \r\nDon't let them make you.. (uh)  \r\nForget your problems (hey)  \r\nForget all your fears (fears)  \r\nWhatever your wishes (wishes) 

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.        , 0.00784614, 0.04053489, ..., 0.01486702, 0.03777535,
       0.00537103])

In [21]:
df[df['song'] == 'Scare Me']

,artist,song,text
3,Kenny Chesney,Scare Me,i 'm not afraid of lone i 've gotten good at b...


In [23]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:5]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [24]:
recommendation('Scare Me')

['Scared To Death', "God's Comic", 'Bodysnatchers', 'The Ghost Psalm']

In [25]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))